In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_mem:.2f} MB")

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage after optimization is: {end_mem:.2f} MB')
    print(f'Decreased by {100 * (start_mem - end_mem) / start_mem:.1f}%')

    return df

In [ ]:
print("Loading files..")
train_transaction = pd.read_csv(r'D:\Lohith\BTECH CODES\Machine_Learning\dataset\test_transaction.csv')
train_identity = pd.read_csv(r'D:\Lohith\BTECH CODES\Machine_Learning\dataset\train_identity.csv')

train_transaction = reduce_mem_usage(train_transaction)

Loading files..


FileNotFoundError: [Errno 2] No such file or directory: 'Machine_Learning\\dataset\\train_transaction.csv'

In [4]:
print("Merging..")
train = pd.merge(train_transaction , train_identity, on="TransactionID", how="left")
del train_transaction,train_identity
gc.collect()
print(f"Datasets Merged, Shape: {train.shape}")

Merging..
Datasets Merged, Shape: (590540, 434)
